# Experiment functionality tutorial with the Investing Algorithm Framework

Welcome to the tutorial about the Experiment functionality within the Investing Algorithm Framework. In this guide, we'll demonstrate how to leverage this feature to conduct A/B Testing effectively. With Experiment, you can effortlessly compare the performance of multiple models, such as challenger versus champion, and experiment with diverse backtest date ranges and parameter configurations.

The Experiment functionality proves invaluable for fine-tuning strategy parameters and comparing different strategies, whether it's testing a new challenger model against the existing production model (champion), or exploring variations in configuration. This process of comparing the challenger model to the production model, or champion, falls under the umbrella of model validation or model evaluation. It entails meticulously assessing the challenger model's performance in relation to the established production model.

## Step 1: Define strategies for experimentation

In [1]:
from investing_algorithm_framework import CCXTOHLCVMarketDataSource, \
    CCXTTickerMarketDataSource


btc_eur_ohlcv_2h_data = CCXTOHLCVMarketDataSource(
    identifier="BTC/EUR_ohlcv_2h",
    symbol="BTC/EUR",
    market="BITVAVO",
    time_frame="2h",
    window_size=200,
)
btc_eur_ticker_data = CCXTTickerMarketDataSource(
    identifier="BTC/EUR_ticker",
    symbol="BTC/EUR",
    market="BITVAVO",
    backtest_time_frame="2h"
)

In [9]:
from investing_algorithm_framework import Algorithm, TradingStrategy, \
    TimeUnit, OrderSide, get_ema, is_below, is_crossover, \
    convert_polars_to_pandas, Context


class AlternativeStrategy(TradingStrategy):
    time_unit = TimeUnit.HOUR
    interval = 2
    market_data_sources = [
        btc_eur_ohlcv_2h_data,
        btc_eur_ticker_data
    ]
    symbols = ["BTC/EUR"]

    def __init__(
        self,
        short_period,
        long_period
    ):
        self.fast = short_period
        self.slow = long_period
        super().__init__()

    def apply_strategy(self, context: Context, market_data):

        for symbol in self.symbols:
            target_symbol = symbol.split('/')[0]

            if context.has_open_orders(target_symbol):
                continue

            polars_df = market_data[f"{symbol}_ohlcv_2h"]
            df = convert_polars_to_pandas(polars_df)
            df = get_ema(data=df, source_column_name="Close", period=self.fast)
            df = get_ema(data=df, source_column_name="Close", period=self.slow)
            ticker_data = market_data[f"{symbol}_ticker"]
            price = ticker_data['bid']

            if not context.has_position(target_symbol) \
                    and is_crossover(
                df, f"EMA_{self.fast}", f"EMA_{self.slow}"
            ):
                order = context.create_limit_order(
                    target_symbol=target_symbol,
                    order_side=OrderSide.BUY,
                    price=price,
                    percentage_of_portfolio=25,
                    precision=4,
                )
                trade = context.get_trade(order_id=order.id)
                trade = context.add_stop_loss(
                    trade=trade,
                    percentage=5,
                    sell_percentage=100,
                    trade_risk_type="trailing"
                )
                trade = context.add_take_profit(
                    trade=trade,
                    percentage=10,
                    sell_percentage=50,
                    trade_risk_type="trailing"
                )
            if context.has_position(target_symbol) \
                    and is_below(
                df, f"EMA_{self.fast}", f"EMA_{self.slow}"
            ):
                open_trades = context.get_open_trades(
                    target_symbol=target_symbol
                )
                for trade in open_trades:
                    context.close_trade(trade)


def create_algorithm(
    name,
    description,
    short_period,
    long_period
) -> Algorithm:
    algorithm = Algorithm(
        name=name,
        description=description
    )
    algorithm.add_strategy(
        AlternativeStrategy(
            short_period,
            long_period,
        )
    )
    return algorithm


In [10]:
from investing_algorithm_framework import Algorithm, TradingStrategy, \
    TimeUnit, OrderSide, get_ema, get_rsi, is_crossover, is_below, \
    convert_polars_to_pandas, Context


class Strategy(TradingStrategy):
    time_unit = TimeUnit.HOUR
    interval = 2
    market_data_sources = [
        btc_eur_ohlcv_2h_data,
        btc_eur_ticker_data
    ]
    symbols = ["BTC/EUR"]

    def __init__(
        self,
        short_period,
        long_period,
        rsi_period,
        rsi_buy_threshold,
        rsi_sell_threshold
    ):
        self.fast = short_period
        self.slow = long_period
        self.rsi_period = rsi_period
        self.rsi_buy_threshold = rsi_buy_threshold
        self.rsi_sell_threshold = rsi_sell_threshold
        super().__init__()

    def apply_strategy(self, context: Context, market_data):

        for symbol in self.symbols:
            target_symbol = symbol.split('/')[0]

            if context.has_open_orders(target_symbol):
                continue

            polars_df = market_data[f"{symbol}_ohlcv_2h"]
            df = convert_polars_to_pandas(polars_df)
            df = get_ema(data=df, source_column_name="Close", period=self.fast)
            df = get_ema(data=df, source_column_name="Close", period=self.slow)
            df = get_rsi(data=df, period=self.rsi_period)
            ticker_data = market_data[f"{symbol}_ticker"]
            price = ticker_data['bid']

            if not context.has_position(target_symbol) \
                    and is_crossover(
                        df, f"EMA_{self.fast}", f"EMA_{self.slow}"
            ) and df[f"RSI_{self.rsi_period}"].iloc[-1] <= self.rsi_buy_threshold:
                order = context.create_limit_order(
                    target_symbol=target_symbol,
                    order_side=OrderSide.BUY,
                    price=price,
                    percentage_of_portfolio=25,
                    precision=4,
                )
                trade = context.get_trade(order_id=order.id)
                trade = context.add_stop_loss(
                    trade=trade,
                    percentage=5,
                    sell_percentage=100,
                    trade_risk_type="trailing"
                )
                trade = context.add_take_profit(
                    trade=trade,
                    percentage=10,
                    sell_percentage=50,
                    trade_risk_type="trailing"
                )

            if context.has_position(target_symbol) \
                    and is_below(
                        df, f"EMA_{self.fast}", f"EMA_{self.slow}"
                    ) and df[f"RSI_{self.rsi_period}"].iloc[-1] > self.rsi_sell_threshold:
                open_trades = context.get_open_trades(
                    target_symbol=target_symbol
                )
                for trade in open_trades:
                    context.close_trade(trade)

def create_alternative_algorithm(
    name,
    description,
    short_period,
    long_period,
    rsi_period,
    rsi_buy_threshold,
    rsi_sell_threshold
) -> Algorithm:
    algorithm = Algorithm(
        name=name,
        description=description
    )
    algorithm.add_strategy(
        Strategy(
            short_period,
            long_period,
            rsi_period,
            rsi_buy_threshold=rsi_buy_threshold,
            rsi_sell_threshold=rsi_sell_threshold
        )
    )
    return algorithm


## Step 2: Setting up the backtest

In [7]:
from investing_algorithm_framework import create_app, BacktestDateRange, \
    CCXTOHLCVMarketDataSource, RESOURCE_DIRECTORY, \
    PortfolioConfiguration, convert_polars_to_pandas
from plotly import graph_objects as go
from pandas import DataFrame as PandasDataFrame
from datetime import datetime

# Define the backtest date ranges
up_turn_date_range = BacktestDateRange(
    start_date=datetime(2022, 12, 20),
    end_date=datetime(2023, 6, 1),
    name="up_turn"
)
sideways_date_range = BacktestDateRange(
    start_date=datetime(2022, 6, 10),
    end_date=datetime(2023, 1, 10),
    name="sideways"
)
down_turn_date_range = BacktestDateRange(
    start_date=datetime(2021, 12, 21),
    end_date=datetime(2022, 6, 20),
    name="down_turn"
)

# Create the app and the champion and challenger algorithms
app = create_app(config={RESOURCE_DIRECTORY: "resources_dump"})
app.add_portfolio_configuration(PortfolioConfiguration(initial_balance=1000, trading_symbol="EUR", market="BITVAVO"))

champion = create_algorithm(
    name="primary",
    description="This is the primary algorithm configured with 21 ema and 50 ema",
    long_period=50,
    short_period=21
)
challenger = create_alternative_algorithm(
    name="secondary",
    description="This is the primary algorithm configured with 21 ema and 50 ema",
    long_period=50,
    short_period=21,
    rsi_period=14,
    rsi_sell_threshold=70,
    rsi_buy_threshold=60
)

def create_prices_graph(
    data: PandasDataFrame, name: str = "Close", color: str = "blue"
) -> go.Scatter:
    return go.Scatter(
        x=data.index,
        y=data['Close'],
        mode='lines',
        line=dict(color=color, width=1),
        name=name
    )

down_turn_data_source = CCXTOHLCVMarketDataSource(
    window_size=100,
    time_frame="2h",
    market="bitvavo",
    symbol="BTC/EUR",
    identifier="BTC/EUR_OHLCV"
)
df_down_turn = down_turn_data_source.get_data(
    start_date=down_turn_date_range.start_date,
    end_date=down_turn_date_range.end_date
)
df_down_turn = convert_polars_to_pandas(df_down_turn)
up_turn_data_source = CCXTOHLCVMarketDataSource(
    window_size=100,
    time_frame="2h",
    market="bitvavo",
    symbol="BTC/EUR",
    identifier="BTC/EUR_OHLCV"
)
df_up_turn = up_turn_data_source.get_data(
    start_date=up_turn_date_range.start_date,
    end_date=up_turn_date_range.end_date
)
df_up_turn = convert_polars_to_pandas(df_up_turn)
side_way_data_source = CCXTOHLCVMarketDataSource(
    window_size=100,
    time_frame="2h",
    market="bitvavo",
    symbol="BTC/EUR",
    identifier="BTC/EUR_OHLCV"
)
df_side_way = side_way_data_source.get_data(
    start_date=sideways_date_range.start_date,
    end_date=sideways_date_range.end_date
)
df_side_way = convert_polars_to_pandas(df_side_way)

from plotly.subplots import make_subplots
fig = make_subplots(rows=3, cols=1, shared_xaxes=False, vertical_spacing=0.02)
fig.add_trace(create_prices_graph(df_down_turn, name="Close down turn", color="red"), row=1, col=1)
fig.add_trace(create_prices_graph(df_up_turn, name="Close up turn", color="green"), row=2, col=1)
fig.add_trace(create_prices_graph(df_side_way, name="Close side ways", color="blue"), row=3, col=1)

fig.update_layout(
    height=600, width=800, title_text="Backtest date range close prices"
)

# Add titles to each subplot
fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_yaxes(title_text="Prices down turn", row=1, col=1)
fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_yaxes(title_text="Prices up turn", row=2, col=1)
fig.update_xaxes(title_text="Date", row=3, col=1)
fig.update_yaxes(title_text="Prices side way", row=3, col=1)
fig.show()

## Step 3: Running the backtests for all the date ranges

In [11]:
reports = app.run_backtests(
    algorithms=[champion, challenger],
    date_ranges=[
        down_turn_date_range, up_turn_date_range, sideways_date_range
    ]
)

Running backtests for date range: down_turn 2021-12-21 00:00:00 - 2022-06-20 00:00:00 for a total of 2 algorithms.


Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 153.90it/s]
Running backtest for algorithm with name primary:   0%|          | 0/2173 [00:00<?, ?it/s]

<SQLTrade(id=1,target_symbol='BTC',trading_symbol='EUR',status='CREATED',amount=0.0057,filled_amount=0.0,remaining=0.0,open_price=43154.0,opened_at=datetime.datetime(2021, 12, 21, 8, 0),closed_at=None,net_gain=0.0,last_reported_price=None)>
<SQLTrade(id=1,target_symbol='BTC',trading_symbol='EUR',status='CREATED',amount=0.0057,filled_amount=0.0,remaining=0.0,open_price=43154.0,opened_at=datetime.datetime(2021, 12, 21, 8, 0),closed_at=None,net_gain=0.0,last_reported_price=None)>


Running backtest for algorithm with name primary:  13%|█▎        | 273/2173 [00:02<00:12, 157.47it/s]

<SQLTrade(id=2,target_symbol='BTC',trading_symbol='EUR',status='CREATED',amount=0.0065,filled_amount=0.0,remaining=0.0,open_price=37933.5,opened_at=datetime.datetime(2022, 1, 12, 10, 0),closed_at=None,net_gain=0.0,last_reported_price=None)>
<SQLTrade(id=2,target_symbol='BTC',trading_symbol='EUR',status='CREATED',amount=0.0065,filled_amount=0.0,remaining=0.0,open_price=37933.5,opened_at=datetime.datetime(2022, 1, 12, 10, 0),closed_at=None,net_gain=0.0,last_reported_price=None)>


...

<SQLTrade(id=4,target_symbol='BTC',trading_symbol='EUR',status='CREATED',amount=0.0065,filled_amount=0.0,remaining=0.0,open_price=37983.5,opened_at=datetime.datetime(2022, 1, 20, 18, 0),closed_at=None,net_gain=0.0,last_reported_price=None)>
<SQLTrade(id=4,target_symbol='BTC',trading_symbol='EUR',status='CREATED',amount=0.0065,filled_amount=0.0,remaining=0.0,open_price=37983.5,opened_at=datetime.datetime(2022, 1, 20, 18, 0),closed_at=None,net_gain=0.0,last_reported_price=None)>


Running backtest for algorithm with name primary:  20%|██        | 437/2173 [00:03<00:13, 131.25it/s]

<SQLTrade(id=5,target_symbol='BTC',trading_symbol='EUR',status='CREATED',amount=0.0074,filled_amount=0.0,remaining=0.0,open_price=33407.0,opened_at=datetime.datetime(2022, 1, 26, 8, 0),closed_at=None,net_gain=0.0,last_reported_price=None)>
<SQLTrade(id=5,target_symbol='BTC',trading_symbol='EUR',status='CREATED',amount=0.0074,filled_amount=0.0,remaining=0.0,open_price=33407.0,opened_at=datetime.datetime(2022, 1, 26, 8, 0),closed_at=None,net_gain=0.0,last_reported_price=None)>


Running backtest for algorithm with name primary:  25%|██▍       | 538/2173 [00:04<00:13, 123.38it/s]


KeyboardInterrupt: 

## Step 4: Analyzing the backtest results

In [18]:
from investing_algorithm_framework import pretty_print_backtest_reports_evaluation, BacktestReportsEvaluation

evaluation = BacktestReportsEvaluation(reports)
pretty_print_backtest_reports_evaluation(evaluation)


              :%%%#+-          .=*#%%%      Backtest reports evaluation
              *%%%%%%%+------=*%%%%%%%-     ---------------------------
              *%%%%%%%%%%%%%%%%%%%%%%%-     Number of reports: 6 backtest reports
              .%%%%%%%%%%%%%%%%%%%%%%#      Largest overall profit: (Algorithm primary) 119.9648 EUR 11.9965% (up_turn 2022-12-20 00:00:00 - 2023-06-01 00:00:00)
               #%%%####%%%%%%%%**#%%%+      Largest overall growth: (Algorithm primary) 125.2748 EUR 12.5275% (up_turn 2022-12-20 00:00:00 - 2023-06-01 00:00:00)
         .:-+*%%%%- -+..#%%%+.+-  +%%%#*=-:
          .:-=*%%%%. += .%%#  -+.-%%%%=-:..
          .:=+#%%%%%*###%%%%#*+#%%%%%%*+-:
                +%%%%%%%%%%%%%%%%%%%=
            :++  .=#%%%%%%%%%%%%%*-
           :++:      :+%%%%%%#-.
          :++:        .%%%%%#=
         :++:        .#%%%%%#*=
        :++-        :%%%%%%%%%+=
       .++-        -%%%%%%%%%%%+=
      .++-        .%%%%%%%%%%%%%+=
     .++-         *%%%%%%%%%%%%%*+:
    .++-  

## Step 5: Highlighting the winning algorithm

In [19]:
from typing import List
import plotly.graph_objects as go
from pandas import DataFrame as PandasDataFrame

from investing_algorithm_framework import Trade

def create_entry_graph(
    data: PandasDataFrame,
    trades: List[Trade],
    column = "Buy",
    value_lookup_column = "Close",
    color = "green"
):

    for trade in trades:
        data.loc[trade.opened_at, column] = 1

    # Filter the data for buy and sell signals
    buy_signals = data[data[column] == 1]

    # Add Buy signals (green circles)
    return go.Scatter(
        x=buy_signals.index,
        y=buy_signals[value_lookup_column],
        mode='markers',
        name='Buy Signal',
        marker=dict(symbol='circle', size=10, color=color)
    )


def create_exit_graph(
    data: PandasDataFrame,
    trades: List[Trade],
    column = "Sell",
    value_lookup_column = "Close",
    color = "red"
):

    for trade in trades:
        data.loc[trade.closed_at, column] = 1

    # Filter the data for buy and sell signals
    buy_signals = data[data[column] == 1]

    # Add Buy signals (green circles)
    return go.Scatter(
        x=buy_signals.index,
        y=buy_signals[value_lookup_column],
        mode='markers',
        name='Sell Signal',
        marker=dict(symbol='circle', size=10, color=color)
    )

In [20]:
from investing_algorithm_framework import pretty_print_backtest, \
    BacktestReportsEvaluation
import pandas as pd

# Set the weights for the evaluation criteria, where we prioritize the profit over the growth
evaluation = BacktestReportsEvaluation(reports)
algorithm_name = evaluation.rank(weight_profit=0.7, weight_growth=0.3)
print(f"The winning algorithm is {algorithm_name}")

# Highlight the winning strategy by displaying the backtest result of the up turn date range
up_turn_report = evaluation.get_report(
    name=algorithm_name, backtest_date_range=up_turn_date_range
)
pretty_print_backtest(up_turn_report)

from plotly.subplots import make_subplots
fig = make_subplots(rows=1, cols=1, shared_xaxes=False, vertical_spacing=0.02)
df_up_turn["Datetime"] = pd.to_datetime(df_up_turn.index)
df_up_turn.set_index('Datetime', inplace=True)

fig.add_trace(create_prices_graph(df_up_turn, name="Close price up turn", color="blue"), row=1, col=1)
fig.add_trace(
    create_entry_graph(df_up_turn, up_turn_report.trades), row=1, col=1
)
fig.add_trace(
    create_exit_graph(df_up_turn, up_turn_report.trades), row=1, col=1
)
fig.update_layout(height=600, width=800, title_text="Up turn backtest date range trades overview for best performing algorithm")
fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_yaxes(title_text="Prices up turn", row=1, col=1)
fig.show()

The winning algorithm is primary

                  :%%%#+-          .=*#%%%        Backtest report
                  *%%%%%%%+------=*%%%%%%%-       ---------------------------
                  *%%%%%%%%%%%%%%%%%%%%%%%-       Start date: 2022-12-20 00:00:00
                  .%%%%%%%%%%%%%%%%%%%%%%#        End date: 2023-06-01 00:00:00
                   #%%%####%%%%%%%%**#%%%+        Number of days: 163
             .:-+*%%%%- -+..#%%%+.+-  +%%%#*=-:   Number of runs: 1957
              .:-=*%%%%. += .%%#  -+.-%%%%=-:..   Number of orders: 39
              .:=+#%%%%%*###%%%%#*+#%%%%%%*+-:    Initial balance: 1000.0
                    +%%%%%%%%%%%%%%%%%%%=         Final balance: 1125.2748
                :++  .=#%%%%%%%%%%%%%*-           Total net gain: 119.9648 12.0%
               :++:      :+%%%%%%#-.              Growth: 125.2748 12.53%
              :++:        .%%%%%#=                Number of trades closed: 1
             :++:        .#%%%%%#*=               Number of trades 

## Step 6: Optimizing the winning strategy with different parameters

In [21]:
from investing_algorithm_framework import pretty_print_backtest_reports_evaluation, BacktestReportsEvaluation

configurations = [
    {
        "name": "primary_21_50",
        "description": "This is the primary algorithm configured with 21 ema and 50 ema",
        "long_period": 50,
        "short_period": 21
    },
    {
        "name": "primary_21_75",
        "description": "This is the primary algorithm configured with 21 ema and 50 ema",
        "long_period": 75,
        "short_period": 21
    },
    {
        "name": "primary_50_100",
        "description": "This is the primary algorithm configured with 21 ema and 50 ema",
        "long_period": 100,
        "short_period": 50
    },
    {
        "name": "primary_50_200",
        "description": "This is the primary algorithm configured with 21 ema and 50 ema",
        "long_period": 200,
        "short_period": 50
    }
]
algorithms = []

for configuration in configurations:
    algorithm = create_algorithm(**configuration)
    algorithms.append(algorithm)

reports = app.run_backtests(algorithms=algorithms, date_ranges=[down_turn_date_range, up_turn_date_range, sideways_date_range])
evaluation = BacktestReportsEvaluation(reports)
pretty_print_backtest_reports_evaluation(evaluation, backtest_date_range=up_turn_date_range)
winning_configuration = evaluation.rank(weight_profit=0.7, weight_growth=0.3, backtest_date_range=up_turn_date_range)
print(f"The winning configuration for the up turn backtest date range is {winning_configuration}")

pretty_print_backtest_reports_evaluation(evaluation, backtest_date_range=sideways_date_range)
winning_configuration = evaluation.rank(weight_profit=0.7, weight_growth=0.3, backtest_date_range=sideways_date_range)
print(f"The winning configuration for the side ways backtest date range is {winning_configuration}")

pretty_print_backtest_reports_evaluation(evaluation, backtest_date_range=down_turn_date_range)
winning_configuration = evaluation.rank(weight_profit=0.7, weight_growth=0.3, backtest_date_range=down_turn_date_range)
print(f"The winning configuration for the down turn backtest date range is {winning_configuration}")

Running backtests for date range: down_turn 2021-12-21 00:00:00 - 2022-06-20 00:00:00 for a total of 4 algorithms.


Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 84.31it/s]
Running backtest for algorithm with name primary_21_50: 100%|██████████| 2173/2173 [00:24<00:00, 88.07it/s] 
Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 184.93it/s]
Running backtest for algorithm with name primary_21_75: 100%|██████████| 2173/2173 [00:23<00:00, 93.05it/s] 
Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 113.39it/s]
Running backtest for algorithm with name primary_50_100: 100%|██████████| 2173/2173 [00:20<00:00, 107.07it/s]
Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 194.65it/s]
Running backtest for algorithm with name primary_50_200: 100%|██████████| 2173/2173 [00:17<00:00, 125.81it/s]


Running backtests for date range: up_turn 2022-12-20 00:00:00 - 2023-06-01 00:00:00 for a total of 4 algorithms.


Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 197.40it/s]
Running backtest for algorithm with name primary_21_50: 100%|██████████| 1957/1957 [00:23<00:00, 83.23it/s] 
Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 132.58it/s]
Running backtest for algorithm with name primary_21_75: 100%|██████████| 1957/1957 [00:21<00:00, 89.83it/s] 
Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 194.67it/s]
Running backtest for algorithm with name primary_50_100: 100%|██████████| 1957/1957 [00:19<00:00, 101.23it/s]
Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 98.48it/s]
Running backtest for algorithm with name primary_50_200: 100%|██████████| 1957/1957 [00:20<00:00, 96.49it/s] 


Running backtests for date range: sideways 2022-06-10 00:00:00 - 2023-01-10 00:00:00 for a total of 4 algorithms.


Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 180.60it/s]
Running backtest for algorithm with name primary_21_50: 100%|██████████| 2569/2569 [00:37<00:00, 69.35it/s] 
Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 100.55it/s]
Running backtest for algorithm with name primary_21_75: 100%|██████████| 2569/2569 [00:31<00:00, 81.22it/s] 
Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 123.20it/s]
Running backtest for algorithm with name primary_50_100: 100%|██████████| 2569/2569 [00:26<00:00, 98.40it/s] 
Preparing backtest market data: 100%|██████████| 2/2 [00:00<00:00, 146.06it/s]
Running backtest for algorithm with name primary_50_200: 100%|██████████| 2569/2569 [00:27<00:00, 94.38it/s] 


              :%%%#+-          .=*#%%%      Backtest reports evaluation
              *%%%%%%%+------=*%%%%%%%-     ---------------------------
              *%%%%%%%%%%%%%%%%%%%%%%%-     Number of reports: 4 backtest reports
              .%%%%%%%%%%%%%%%%%%%%%%#      Largest overall profit: (Algorithm primary_21_75) 122.7062 EUR 12.2706% (up_turn 2022-12-20 00:00:00 - 2023-06-01 00:00:00)
               #%%%####%%%%%%%%**#%%%+      Largest overall growth: (Algorithm primary_21_75) 128.2862 EUR 12.8286% (up_turn 2022-12-20 00:00:00 - 2023-06-01 00:00:00)
         .:-+*%%%%- -+..#%%%+.+-  +%%%#*=-:
          .:-=*%%%%. += .%%#  -+.-%%%%=-:..
          .:=+#%%%%%*###%%%%#*+#%%%%%%*+-:
                +%%%%%%%%%%%%%%%%%%%=
            :++  .=#%%%%%%%%%%%%%*-
           :++:      :+%%%%%%#-.
          :++:        .%%%%%#=
         :++:        .#%%%%%#*=
        :++-        :%%%%%%%%%+=
       .++-        -%%%%%%%%%%%+=
      .++-        .%%%%%%%%%%%%%+=
     .++-         *%%%%%%%%%%%%%*+

## Conclusion

In this guide we have demonstrated how to leverage the backtests functionality within the Investing Algorithm Framework to conduct A/B Testing effectively. We have compared the performance of multiple models, such as challenger versus champion, and experimented with diverse backtest date ranges and parameter configurations. This process of comparing the challenger model to the production model, or champion, falls under the umbrella of model validation or model evaluation. It entails meticulously assessing the challenger model's performance in relation to the established production model. The Experiment functionality proves invaluable for fine-tuning strategy parameters and comparing different strategies, whether it's testing a new challenger model against the existing production model (champion), or exploring variations in configuration.

Also, when selected an algorithm we showed how to test different configurations of the winning algorithm to optimize the strategy. This process is crucial to ensure that the algorithm is performing at its best and to maximize the profit. By following these steps, you can effectively compare different algorithms and configurations to identify the best performing strategy for your trading needs.